In [ ]:
!git clone -b kob_2020 https://github.com/HamidrezaAmirzadeh/Ours.git
# !git clone https://github.com/HamidrezaAmirzadeh/Ours.git

In [ ]:
!pip install mosestokenizer -q
!pip install sacremoses -q
!pip install fastBPE -q

In [ ]:
"""Before running this code, make sure of your modifications to the fairseq, because
I couldn't manage to get importlib.reload to work and reload the installed fairseq
"""
%cd Ours/fairseq_latest/
!pip install -e.

In [ ]:
import torch
# import importlib
# importlib.reload(fairseq.models.transformer)
from fairseq.models.transformer import TransformerModel

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Download published trained model (2.79GB)
!wget "https://dl.fbaipublicfiles.com/fairseq/models/wmt19.de-en.joined-dict.single_model.tar.gz"

In [6]:
# Load trained model

path = './wmt19.de-en.joined-dict.single_model.tar.gz'

de2en = TransformerModel.from_pretrained(
    model_name_or_path=path,
    tokenizer='moses',
    bpe='fastbpe',
).to(device)

de2en.eval()
print()

In [7]:
de = "Wir glauben nicht, daß wir nur Rosinen herauspicken sollten."
en = "We do not believe that we should cherry-pick."

eos_id = de2en.tgt_dict.index("</s>")

# Get indices and tokenized text
print("De (source)")
print("Raw text:", de)
de_idx = de2en.encode(de).unsqueeze(0).to(device)
print("Indices:", de_idx)
de_toks = [de2en.src_dict[i] for i in de_idx[0]]
print("Tokenized text:", de_toks)

print("\nEn (target)")
print("Raw text:", en)
en_idx = torch.tensor([eos_id] + [de2en.tgt_dict.index(t) for t in de2en.bpe.encode(en).split()]).unsqueeze(0).to(device)
print("Indices:", en_idx)
en_toks = [de2en.tgt_dict[i] for i in en_idx[0]]
print("Tokenized text:", en_toks)

De (source)
Raw text: Wir glauben nicht, daß wir nur Rosinen herauspicken sollten.
Indices: tensor([[  396,  7952,    74,     4,   514,   150,   249,  2759,  4089,   730,
           640,   393, 15810,  1882,     5,     2]])
Tokenized text: ['Wir', 'glauben', 'nicht', ',', 'daß', 'wir', 'nur', 'Ros@@', 'inen', 'her@@', 'au@@', 'sp@@', 'icken', 'sollten', '.', '</s>']

En (target)
Raw text: We do not believe that we should cherry-pick.
Indices: tensor([[    2,   333,   283,    90,  2989,    38,   136,   505,   199,  5939,
           167,   406, 21075,     5]])
Tokenized text: ['</s>', 'We', 'do', 'not', 'believe', 'that', 'we', 'should', 'ch@@', 'err@@', 'y@@', '-@@', 'pick@@', '.']


In [10]:
with torch.no_grad():
    outputs = de2en.models[0](de_idx, de_idx.size()[-1], en_idx, output_all_attentions=True)

In [22]:
# save the version of the libraries used in this environment
!pip freeze > requirements.txt
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>